# Brain Tumor MRI Classification Project

## 1 Project Overview

## Extract Preprocessed Data

In [ ]:
import os
import zipfile
import glob

def extract_preprocessed_data():
    zip_candidates = ['/content/preprocessed_data.zip', *glob.glob('/content/*preprocessed*.zip')]
    zip_path = None
    for candidate in zip_candidates:
        if os.path.exists(candidate):
            zip_path = candidate
            break

    if not zip_path:
        print("preprocessed_data.zip not found in /content/")
        return False

    if os.path.exists('/content/preprocessed_data') and os.path.exists('/content/preprocessed_data/config.json'):
        required_files = [
            'X_train.npy', 'X_val.npy', 'X_test.npy',
            'y_train.npy', 'y_val.npy', 'y_test.npy',
            'y_train_cat.npy', 'y_val_cat.npy', 'y_test_cat.npy',
            'config.json'
        ]
        missing = [f for f in required_files if not os.path.exists(f'/content/preprocessed_data/{f}')]
        if not missing:
            print("preprocessed_data folder already exists")
            return True

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('/content/')

        print(f"Extraction completed: {os.path.basename(zip_path)}")
        return True
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return False

if extract_preprocessed_data():
    for f in sorted(os.listdir('/content/preprocessed_data')):
        print(f"├── {f}")
else:
    print("Cannot proceed without preprocessed data")

Extraction completed: preprocessed_data.zip
├── X_test.npy
├── X_train.npy
├── X_val.npy
├── config.json
├── y_test.npy
├── y_test_cat.npy
├── y_train.npy
├── y_train_cat.npy
├── y_val.npy
├── y_val_cat.npy


## Environment and Dependencies
We utilize TensorFlow and Keras for building the neural network, along with NumPy and Pandas
for data handling. Matplotlib and Seaborn are used for performance visualization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler)
from tensorflow.keras.optimizers import Adam
np.random.seed(42)
tf.random.set_seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Data Pipeline
The dataset consists of preprocessed MRI scans stored as NumPy arrays. We define paths for
loading data and saving training artifacts.

In [ ]:
DATA_PATH = '/content/preprocessed_data'
OUTPUT_PATH = '/content/training_results'
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(f'{OUTPUT_PATH}/models', exist_ok=True)
os.makedirs(f'{OUTPUT_PATH}/histories', exist_ok=True)
os.makedirs(f'{OUTPUT_PATH}/plots', exist_ok=True)
X_train = np.load(f'{DATA_PATH}/X_train.npy')
X_val = np.load(f'{DATA_PATH}/X_val.npy')
X_test = np.load(f'{DATA_PATH}/X_test.npy')
y_train_cat = np.load(f'{DATA_PATH}/y_train_cat.npy')
y_val_cat = np.load(f'{DATA_PATH}/y_val_cat.npy')
y_test_cat = np.load(f'{DATA_PATH}/y_test_cat.npy')
with open(f'{DATA_PATH}/config.json', 'r') as f:
    config = json.load(f)

## Data Augmentation Strategy

To improve model generalization and mitigate overfitting, we implement a moderate augmentation strategy that includes rotations, shifts, and flips. Vertical flipping is deemed safe for MRI
brain scans.

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

## Model Architecture

The optimized CNN architecture consists of four convolutional blocks with increasing filter sizes
(32, 64, 128, 256). Each block is followed by Batch Normalization and Dropout.

In [ ]:
def build_optimized_cnn(input_shape=(224, 224, 3), num_classes=4):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.35),
        # Block 4
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.35),
        # Dense classification head
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.55),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.55),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
model = build_optimized_cnn(
    input_shape=X_train.shape[1:],
    num_classes=config['num_classes']
)

## Training Process

The model is trained for 100 epochs using the Adam optimizer. We monitor validation accuracy
to save the best weights and reduce the learning rate when the loss plateaus.

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
callbacks = [
    ModelCheckpoint(filepath=f'{OUTPUT_PATH}/models/best_model.h5', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7)
]

In [ ]:
history = model.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=32),
    epochs=100,
    validation_data=(X_val, y_val_cat),
    callbacks=callbacks
)

Epoch 1/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 551ms/step - accuracy: 0.5085 - loss: 1.4513 - precision: 0.5448 - recall: 0.4590

152/152 ━━━━━━━━━━━━━━━━━━━━ 130s 627ms/step - accuracy: 0.5090 - loss: 1.4496 - precision: 0.5453 - recall: 0.4594 - val_accuracy: 0.2835 - val_loss: 1.7722 - val_precision: 0.2923 - val_recall: 0.2660 - learning_rate: 0.0010
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 64s 423ms/step - accuracy: 0.6674 - loss: 0.8960 - precision: 0.7062 - recall: 0.6142 - val_accuracy: 0.2789 - val_loss: 3.0125 - val_precision: 0.2789 - val_recall: 0.2789 - learning_rate: 0.0010
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 422ms/step - accuracy: 0.6916 - loss: 0.7933 - precision: 0.7347 - recall: 0.6363 - val_accuracy: 0.2789 - val_loss: 3.1869 - val_precision: 0.2789 - val_recall: 0.2789 - learning_rate: 0.0010
Epoch 4/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.7412 - loss: 0.6602 - precision: 0.7754 - recall: 0.6994

152/152 ━━━━━━━━━━━━━━━━━━━━ 67s 436ms/step - accuracy: 0.7412 - loss: 0.6601 - precision: 0.7755 - recall: 0.6995 - val_accuracy: 0.5951 - val_loss: 1.6923 - val_precision: 0.5995 - val_recall: 0.5904 - learning_rate: 0.0010
Epoch 5/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step - accuracy: 0.7568 - loss: 0.6168 - precision: 0.7846 - recall: 0.7135

152/152 ━━━━━━━━━━━━━━━━━━━━ 73s 477ms/step - accuracy: 0.7569 - loss: 0.6166 - precision: 0.7847 - recall: 0.7136 - val_accuracy: 0.7713 - val_loss: 0.7189 - val_precision: 0.7724 - val_recall: 0.7643 - learning_rate: 0.0010
Epoch 6/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 64s 419ms/step - accuracy: 0.7858 - loss: 0.5751 - precision: 0.8206 - recall: 0.7518 - val_accuracy: 0.6826 - val_loss: 1.2980 - val_precision: 0.6864 - val_recall: 0.6768 - learning_rate: 0.0010
Epoch 7/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 66s 429ms/step - accuracy: 0.8263 - loss: 0.4709 - precision: 0.8443 - recall: 0.7951 - val_accuracy: 0.3314 - val_loss: 5.0027 - val_precision: 0.3310 - val_recall: 0.3302 - learning_rate: 0.0010
Epoch 8/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 425ms/step - accuracy: 0.8400 - loss: 0.4350 - precision: 0.8604 - recall: 0.8148 - val_accuracy: 0.7596 - val_loss: 0.7591 - val_precision: 0.7672 - val_recall: 0.7538 - learning_rate: 0.0010
Epoch 9/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - acc

152/152 ━━━━━━━━━━━━━━━━━━━━ 73s 483ms/step - accuracy: 0.8362 - loss: 0.4492 - precision: 0.8524 - recall: 0.8109 - val_accuracy: 0.8436 - val_loss: 0.4396 - val_precision: 0.8601 - val_recall: 0.8320 - learning_rate: 0.0010
Epoch 10/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 64s 420ms/step - accuracy: 0.8406 - loss: 0.4191 - precision: 0.8601 - recall: 0.8250 - val_accuracy: 0.7550 - val_loss: 0.7010 - val_precision: 0.7703 - val_recall: 0.7316 - learning_rate: 0.0010
Epoch 11/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 427ms/step - accuracy: 0.8523 - loss: 0.3896 - precision: 0.8640 - recall: 0.8334 - val_accuracy: 0.7526 - val_loss: 0.7498 - val_precision: 0.7577 - val_recall: 0.7480 - learning_rate: 0.0010
Epoch 12/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 429ms/step - accuracy: 0.8473 - loss: 0.4074 - precision: 0.8629 - recall: 0.8301 - val_accuracy: 0.7316 - val_loss: 0.6804 - val_precision: 0.7745 - val_recall: 0.7013 - learning_rate: 0.0010
Epoch 13/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 64s 423ms/step 

152/152 ━━━━━━━━━━━━━━━━━━━━ 70s 463ms/step - accuracy: 0.8949 - loss: 0.2785 - precision: 0.9052 - recall: 0.8878 - val_accuracy: 0.8926 - val_loss: 0.2802 - val_precision: 0.9068 - val_recall: 0.8856 - learning_rate: 5.0000e-04
Epoch 22/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.9011 - loss: 0.2549 - precision: 0.9113 - recall: 0.8941

152/152 ━━━━━━━━━━━━━━━━━━━━ 71s 463ms/step - accuracy: 0.9011 - loss: 0.2549 - precision: 0.9113 - recall: 0.8941 - val_accuracy: 0.8996 - val_loss: 0.2891 - val_precision: 0.9106 - val_recall: 0.8915 - learning_rate: 5.0000e-04
Epoch 23/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 427ms/step - accuracy: 0.9271 - loss: 0.2054 - precision: 0.9333 - recall: 0.9202 - val_accuracy: 0.8355 - val_loss: 0.4888 - val_precision: 0.8556 - val_recall: 0.8296 - learning_rate: 5.0000e-04
Epoch 24/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 426ms/step - accuracy: 0.9171 - loss: 0.2294 - precision: 0.9247 - recall: 0.9105 - val_accuracy: 0.5204 - val_loss: 1.5668 - val_precision: 0.5576 - val_recall: 0.4854 - learning_rate: 5.0000e-04
Epoch 25/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 426ms/step - accuracy: 0.9069 - loss: 0.2362 - precision: 0.9148 - recall: 0.8988 - val_accuracy: 0.8740 - val_loss: 0.4000 - val_precision: 0.8868 - val_recall: 0.8681 - learning_rate: 5.0000e-04
Epoch 26/100
152/152 ━━━━━━━━━━━━━━━━━━━━

152/152 ━━━━━━━━━━━━━━━━━━━━ 74s 487ms/step - accuracy: 0.9288 - loss: 0.2007 - precision: 0.9341 - recall: 0.9268 - val_accuracy: 0.9417 - val_loss: 0.2001 - val_precision: 0.9448 - val_recall: 0.9393 - learning_rate: 2.5000e-04
Epoch 30/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 72s 421ms/step - accuracy: 0.9410 - loss: 0.1641 - precision: 0.9464 - recall: 0.9367 - val_accuracy: 0.9032 - val_loss: 0.2814 - val_precision: 0.9087 - val_recall: 0.8938 - learning_rate: 2.5000e-04
Epoch 31/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 63s 416ms/step - accuracy: 0.9338 - loss: 0.1781 - precision: 0.9373 - recall: 0.9278 - val_accuracy: 0.8343 - val_loss: 0.5308 - val_precision: 0.8430 - val_recall: 0.8273 - learning_rate: 2.5000e-04
Epoch 32/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 63s 412ms/step - accuracy: 0.9366 - loss: 0.1645 - precision: 0.9425 - recall: 0.9319 - val_accuracy: 0.8926 - val_loss: 0.3163 - val_precision: 0.8963 - val_recall: 0.8880 - learning_rate: 2.5000e-04
Epoch 33/100
152/152 ━━━━━━━━━━━━━━━━━━━━

152/152 ━━━━━━━━━━━━━━━━━━━━ 74s 484ms/step - accuracy: 0.9590 - loss: 0.1052 - precision: 0.9614 - recall: 0.9576 - val_accuracy: 0.9428 - val_loss: 0.2148 - val_precision: 0.9449 - val_recall: 0.9405 - learning_rate: 3.1250e-05
Epoch 53/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step - accuracy: 0.9656 - loss: 0.0969 - precision: 0.9682 - recall: 0.9651

152/152 ━━━━━━━━━━━━━━━━━━━━ 67s 441ms/step - accuracy: 0.9656 - loss: 0.0969 - precision: 0.9682 - recall: 0.9651 - val_accuracy: 0.9463 - val_loss: 0.1997 - val_precision: 0.9495 - val_recall: 0.9428 - learning_rate: 3.1250e-05
Epoch 54/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 65s 426ms/step - accuracy: 0.9618 - loss: 0.1040 - precision: 0.9632 - recall: 0.9599 - val_accuracy: 0.9323 - val_loss: 0.2488 - val_precision: 0.9344 - val_recall: 0.9312 - learning_rate: 3.1250e-05
Epoch 55/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 66s 435ms/step - accuracy: 0.9657 - loss: 0.1005 - precision: 0.9682 - recall: 0.9632 - val_accuracy: 0.9452 - val_loss: 0.1885 - val_precision: 0.9461 - val_recall: 0.9417 - learning_rate: 3.1250e-05
Epoch 56/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 67s 443ms/step - accuracy: 0.9575 - loss: 0.1136 - precision: 0.9627 - recall: 0.9556 - val_accuracy: 0.9323 - val_loss: 0.2424 - val_precision: 0.9354 - val_recall: 0.9300 - learning_rate: 3.1250e-05
Epoch 57/100
152/152 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
MODEL_NAME = 'custom_cnn'

In [ ]:
history_path = f'{OUTPUT_PATH}/histories/{MODEL_NAME}_history.npy'
np.save(history_path, history.history)

In [ ]:
final_model_path = f'{OUTPUT_PATH}/models/{MODEL_NAME}_final.h5'
model.save(final_model_path)

## Test Time Augmentation (TTA)

TTA is utilized during the inference phase. By generating 10 augmented versions of each test
image and averaging the predictions, we significantly increase the robustness of the final classification.

In [ ]:
best_model = keras.models.load_model(f'{OUTPUT_PATH}/models/{MODEL_NAME}_final.h5')

In [ ]:
def predict_with_tta(model, X, n_augmentations=10):
    predictions = []
    preds = model.predict(X, verbose=0)
    predictions.append(preds)
    tta_gen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True
    )
    for i in range(n_augmentations):
        aug_iterator = tta_gen.flow(X, batch_size=len(X), shuffle=False)
        X_aug = next(iter(aug_iterator))
        preds_aug = model.predict(X_aug, verbose=0)
        predictions.append(preds_aug)
    return np.mean(predictions, axis=0)

test_preds_tta = predict_with_tta(best_model, X_test, n_augmentations=10)
test_acc_tta = np.mean(np.argmax(test_preds_tta, axis=1) == np.argmax(y_test_cat, axis=1))

print(f"\nTTA completed!")

# EVALUATION

print("\n")
print("EVALUATION")

# Validation
val_results = best_model.evaluate(X_val, y_val_cat, verbose=0)
print(f"\nValidation Results (Best Model):")
print(f"Loss: {val_results[0]:.4f}")
print(f"Accuracy: {val_results[1]*100:.2f}%")
print(f"Precision: {val_results[2]:.4f}")
print(f"Recall: {val_results[3]:.4f}")

# Test (standard)
test_results = best_model.evaluate(X_test, y_test_cat, verbose=0)
print(f"\nTest Results (Standard):")
print(f"Loss: {test_results[0]:.4f}")
print(f"Accuracy: {test_results[1]*100:.2f}%")
print(f"Precision: {test_results[2]:.4f}")
print(f"Recall: {test_results[3]:.4f}")

# Test (with TTA)
print(f"\nTest Results (With TTA):")
print(f"Accuracy: {test_acc_tta*100:.2f}%")

print("\nSUMMARY:")
print(f"Baseline Test Acc: 93.82%")
print(f"Test Acc (Standard): {test_results[1]*100:.2f}%")
print(f"Test Acc (TTA): {test_acc_tta*100:.2f}%")


TTA completed!


EVALUATION

Validation Results (Best Model):
Loss: 0.2448
Accuracy: 93.70%
Precision: 0.9368
Recall: 0.9335

Test Results (Standard):
Loss: 0.2143
Accuracy: 93.14%
Precision: 0.9353
Recall: 0.9260

Test Results (With TTA):
Accuracy: 97.71%

SUMMARY:
Baseline Test Acc: 93.82%
Test Acc (Standard): 93.14%
Test Acc (TTA): 97.71%
